In [ ]:
import nibabel as nib
import numpy as np
import pandas as pd
from pathlib import Path
from glob import glob

class NiftiCenterExtractor:
    """Glob NIfTI files and record their physical centre coordinates."""
    def __init__(self, pattern: str = "**/*.nii*") -> None:
        self.files = glob(pattern)

    def _center_coordinate(self, nii_path: Path) -> list[float]:
        img = nib.load(str(nii_path))
        nx, ny, nz = img.shape[:3]
        voxel_center = np.array([(nx - 1) / 2, (ny - 1) / 2, (nz - 1) / 2], dtype=float)
        world_center = nib.affines.apply_affine(img.affine, voxel_center)
        return world_center.tolist()

    def to_dataframe(self) -> pd.DataFrame:
        rows = [{"subject": str(p), "coordinate": self._center_coordinate(p)} for p in self.files]
        return pd.DataFrame(rows)

In [ ]:
# Run extractor on the uploaded file
extractor = NiftiCenterExtractor("/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/AD_dataset/*/stimulations/MNI_ICBM_2009b_NLIN_ASYM/gs_20180403170745/vat_efield_left.nii")
df = extractor.to_dataframe()
df

In [ ]:
extractor.files

In [ ]:
df_concat = pd.concat([df, df2], axis=0, ignore_index=True)
df_concat

In [ ]:
df_concat.to_csv('/Volumes/HowExp/datasets/03h_ADVANCE_Alzheimer_DBS/metadata/stim_centroids.csv', index=False)

In [ ]:
def extract_subid_session(subject_path):
    # Split after '/subjects/' and get the next directory as subid
    parts = subject_path.split('/subjects/')
    if len(parts) > 1:
        rest = parts[1]
        subid = rest.split('/', 1)[0]
        # Get the filename as session (last part after '/')
        session = subject_path.split('/')[-1].split('.nii')[0].split('efield_')[1]
    else:
        subid = None
        session = None
    return pd.Series({'Subid': subid, 'Session': session})

df_concat[['Subid', 'Session']] = df_concat['subject'].apply(extract_subid_session)
df_concat